# Få tak i værdata

In [24]:
import openai
import json
import os

openai.api_key = os.getenv("OPENAI_API_KEY")

messages = [
  {
    "role": "system",
    "content": "You are a weather reporter.",
  },
  {
    "role": "user",
    "content": "Hvordan er været i Oslo? Og hva med Trondheim? Og hva er temperaturforskjellen mellom de to byene?",
  }
]

functions = [
  {
    "name": "get_weather",
    "description": "Get the weather for a given location.",
    "parameters": {
      "type": "object",
      "properties": {
        "location": {
          "type": "string",
          "description": "The location to get the weather for.",
        }
      },
      "required": ["location"],
    }
  }
]

def get_weather(location):
  import requests
  response = requests.get(f"https://wttr.in/{location}?format=j1").json()
  return json.dumps(response["current_condition"][0])

function_calls = {
  "get_weather": lambda args: get_weather(args["location"])
}

completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
)

i = 0
while True:
  if i > 10:
    print("Too many function calls.")
    break
  i += 1

  if completion.choices[0].finish_reason == "stop":
    print()
    print("Response:")
    print(completion.choices[0].message.content)
    break

  if completion.choices[0].finish_reason == "function_call":
    function_name = completion.choices[0].message.function_call.name
    function_args = completion.choices[0].message.function_call.arguments
    print(f"Calling function: \"{completion.choices[0].message.function_call.name}\" with parameters: " + str(function_args).replace("\n", " ").replace("   ", " "))
    value = function_calls[function_name](json.loads(function_args))
    print("Sending function return value back to GPT")
    print()

    messages.append({
        "role": "assistant",
        "content": None,
        "function_call": {
            "name": function_name,
            "arguments": function_args,
        },
    })

    messages.append({
        "role": "function",
        "content": str(value),
        "name": function_name,
    })

    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        temperature=0,
    )


Calling function: "get_weather" with parameters: { "location": "Oslo" }
Sending function value back to GPT

Calling function: "get_weather" with parameters: { "location": "Trondheim" }
Sending function value back to GPT


Response:
I Oslo er det nå klart vær med en temperatur på 13°C (55°F). Vinden blåser fra øst-nordøst med en hastighet på 13 km/t (8 mph). Luftfuktigheten er på 88%, og det er en siktbarhet på 2 km (1 mile).

I Trondheim er det derimot overskyet vær med en temperatur på 11°C (52°F). Vinden blåser fra øst med en hastighet på 9 km/t (6 mph). Luftfuktigheten er på 100%, og det er en siktbarhet på 10 km (6 miles).

Temperaturforskjellen mellom Oslo og Trondheim er derfor 2°C (3.6°F), med Oslo som er litt varmere.
